In [2]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.svm import SVC

In [10]:
np.random.seed(0)
data = np.random.rand(800, 2)

In [12]:
X = rbf_kernel(data, data, 1.8)

In [31]:
# make some fake data X and y
np.random.seed(0)
train_X = np.random.rand(100, 3)
train_y = np.random.randint(0,10, size=100)
# print(train_y.min(), train_y.max())
svc = SVC()
svc.fit(train_X, train_y)

SVC()

In [32]:
test_X = np.random.rand(1, 3)
svc.predict(test_X)

array([2])